In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
import cv2
import os
from keras.utils import to_categorical
from keras.optimizers import SGD,RMSprop, adadelta, Adam

Using TensorFlow backend.


In [2]:
PATH = os.getcwd()
data_path = PATH + './mnist_png/training'
data_Dir_list = os.listdir(data_path)
print(data_Dir_list)
img_row = 28
img_col = 28
num_channel = 1
epoch = 20
num_classes = 10

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [3]:
img_data_list = []
images = []
i = 0

for dataset in data_Dir_list:
    img_list = os.listdir(data_path + '/' + dataset)
    for img in img_list:
        i = i+1
        ip_img = cv2.imread(data_path + '/' + dataset + '/' + img)
        ip_img = cv2.cvtColor(ip_img, cv2.COLOR_BGR2GRAY)
        ip_img = cv2.resize(ip_img, (img_row,img_col))
        img_data_list.append(ip_img)
    images.append(i)

print(images)

[6903, 14780, 21770, 28911, 35735, 42048, 48924, 56217, 63042, 70000]


In [4]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
img_data.shape

(70000, 28, 28)

In [5]:
#Expand dimesnsions
img_data= np.expand_dims(img_data, axis=4)
img_data.shape

C:\Users\Praneet\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  from ipykernel import kernelapp as app


(70000, 28, 28, 1)

In [6]:
#Preprocessing
n_samples = img_data.shape[0]
labels = np.ones((n_samples,), dtype='int64')

label_dictionary = {0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6:'6', 7:'7', 8:'8', 9:'9'}

labels[:images[0]] = 0
labels[images[0]:images[1]] = 1
labels[images[1]:images[2]] = 2
labels[images[2]:images[3]] = 3
labels[images[3]:images[4]] = 4
labels[images[4]:images[5]] = 5
labels[images[5]:images[6]] = 6
labels[images[6]:images[7]] = 7
labels[images[7]:images[8]] = 8
labels[images[8]:images[9]] = 9

#One-Hot encoded format
Y = to_categorical(labels, num_classes)

In [7]:
Y

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [8]:
#Shuffle data
from sklearn.utils import shuffle
x, y = shuffle(img_data, Y, random_state=2)
#Split Dataset
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.25, random_state=2)

In [9]:
tf.reset_default_graph()

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(img_row, img_col, 1)))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, (2,2), activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 5, 128)         32896     
__________

In [11]:
model.fit(X_train, Y_train, batch_size=32, epochs=epoch, validation_data=(X_test, Y_test), verbose=1)

Train on 52500 samples, validate on 17500 samples
Epoch 1/20
52500/52500 [==============================] - 70s 1ms/step - loss: 0.4726 - acc: 0.8467 - val_loss: 0.0838 - val_acc: 0.9750
Epoch 2/20
52500/52500 [==============================] - 46s 884us/step - loss: 0.1501 - acc: 0.9581 - val_loss: 0.0586 - val_acc: 0.9832
Epoch 3/20
52500/52500 [==============================] - 46s 874us/step - loss: 0.1233 - acc: 0.9655 - val_loss: 0.0515 - val_acc: 0.9846
Epoch 4/20
52500/52500 [==============================] - 46s 878us/step - loss: 0.1035 - acc: 0.9717 - val_loss: 0.0450 - val_acc: 0.9868
Epoch 5/20
52500/52500 [==============================] - 46s 871us/step - loss: 0.0984 - acc: 0.9746 - val_loss: 0.0428 - val_acc: 0.9882
Epoch 6/20
52500/52500 [==============================] - 46s 868us/step - loss: 0.0894 - acc: 0.9758 - val_loss: 0.0419 - val_acc: 0.9887
Epoch 7/20
52500/52500 [==============================] - 46s 871us/step - loss: 0.0840 - acc: 0.9771 - val_loss: 0.04

In [12]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.029736063707
Test accuracy: 0.9916


In [13]:
target_dir = './newmodels/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./newmodels/model.h5')
model.save_weights('./newmodels/weights.h5')

In [14]:
import numpy as np
from keras.preprocessing import image
#test_image = image.load_img('43.png', target_size=(28,28))
ip_img = cv2.imread('43.png')
ip_img = cv2.cvtColor(ip_img, cv2.COLOR_BGR2GRAY)
ip_img = cv2.resize(ip_img, (img_row,img_col))
test_image = image.img_to_array(ip_img)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict_proba(test_image)
'''
number = 0
for i in result[0]:
    if i == 1:
        result = number
    number += 1
'''
#print("The Number is = %d" % result )
print(result)

TypeError: %d format: a number is required, not numpy.ndarray